In [27]:
# Verificar se melhoria de resultados na divulgaçao performam melhor que o IBOVESPA
# Acessar mongo e pegar dados de divulgação por empresa
# verificar se houve melhoria de algum indicador específico
# se houve, verificar qual a rentabilidade da empresa até a próxima divulgação de resultados ou início da divulgação do terceiro trimestre
    # pegar os preços dos ativos no mysql

# _id: dia em string 27/07/2023
# _129283 = chave e o dicionario abaixo é o valor
# empresa
# "VALE S.A."
# nr_sequencial
# 129283
# descricao
# "Informações Trimestrais - 30/06/2023 - V1"
# data_doc
# "30/06/2023"
# data_envio
# "27/07/2023"
# hora_envio
# "27/07/2023 19:48:48"
# link_consulta
# "https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequen…"
# link_download
# "http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?Codig…"

In [ ]:
def doc_divulgacaoitr():


In [53]:
from pymongo import MongoClient
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

def finddoc_byequity(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byid(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(db, collection_name, containing_word):
    count = 0
    while count != 1:
        collection = db[collection_name]
        regex = re.compile(containing_word, re.IGNORECASE)
        documents = collection.find({'_id': regex})
        count = collection.count_documents({'_id': regex})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            containing_word = input("Qual das empresas seria a solicitada: ")
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

In [59]:
stock_list = ['ROMI3', 'WEGE3', 'ASAI3']

collection = 'divulgacaoinfo_resultados'

for stock in stock_list:

    document = finddoc_byequity(db, 'Empresas_fin6', stock)
    df1 = document.next() # dicionario referente a stock pick

    id_empresa = df1["_id"]    
    print(f'Acao: {stock}')
    print(f'_id : {id_empresa}') 

    document = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa)
    df2 = document.next() # dicionario referente a stock pick

    id_divulgacao = df2["nome_empresa"]    
    print(f'_id : {id_divulgacao}')
    print(f'cod_cvm: {df2["cod_cvm"]}')
    print(f'cod_cvm: {df2["data_envio"]}')    
    print(f'hora_envio: {df2["hora_envio"]}\n')


# for stock in stock_list:
#     document = finddoc_byequity(db, 'Empresas_fin6', stock)
#     df1 = document.next() # dicionario referente a stock pick

#     id_empresa = df1["_id"]    
#     print(f'Acao: {stock}')
#     print(f'_id : {id_empresa}\n') 

# document = finddoc_byequity(db, 'Empresas_fin6', stock_list[0])
# df1 = document.next() # dicionario referente a stock pick

# valor_cod_cvm = df1["_id"]
# print(valor_cod_cvm)

Acao: ROMI3
_id : ROMI S.A.
_id : ROMI S.A.
cod_cvm: 7510
cod_cvm: 18/07/2023
hora_envio: 18/07/2023 18:08:09

Acao: WEGE3
_id : WEG S.A.
_id : WEG S.A.
cod_cvm: 5410
cod_cvm: 19/07/2023
hora_envio: 19/07/2023 07:03:26

Acao: ASAI3
_id : SENDAS DISTRIBUIDORA S.A.
_id : SENDAS DISTRIBUIDORA S.A.
cod_cvm: 25372
cod_cvm: 26/07/2023
hora_envio: 26/07/2023 19:32:29



In [ ]:
db = cliente.CVM
collection = 'Empresas_fin6'

In [42]:
trimestres = []
for chave, valor in df1.items():
    if isinstance(chave, str):
        trimestres.append(chave)

In [40]:
dict_DFCons = df1['2023-06-30']
print(dict_DFCons.items())

for item in dict_DFCons.items():
    print(item)

dict_items([('DFConsolidado', {'DF Consolidado - Balanço Patrimonial Ativo': {'1': {'st_conta': 'True', 'ds_conta': 'Ativo Total', 'vl_conta': 2075762530.0}, '1.01': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 65260363.0}, '1.01.01': {'st_conta': 'True', 'ds_conta': 'Caixa', 'vl_conta': 22541168.0}, '1.01.02': {'st_conta': 'True', 'ds_conta': 'Aplicações de Liquidez', 'vl_conta': 42719195.0}, '1.02': {'st_conta': 'True', 'ds_conta': 'Ativos Financeiros', 'vl_conta': 1854116410.0}, '1.02.01': {'st_conta': 'True', 'ds_conta': 'Depósito Compulsório Banco Central', 'vl_conta': 97135227.0}, '1.02.02': {'st_conta': 'True', 'ds_conta': 'Ativos Financeiros Avaliados ao Valor Justo através do Resultado', 'vl_conta': 16631734.0}, '1.02.02.01': {'st_conta': 'True', 'ds_conta': 'Títulos e Valores Mobiliários', 'vl_conta': 11752639.0}, '1.02.02.02': {'st_conta': 'True', 'ds_conta': 'Derivativos', 'vl_conta': 4879095.0}, '1.02.03': {'st_conta': 'True', 'ds_conta': '

In [37]:
trimestres

['_id',
 'cod_cvm',
 'cnpj_cia',
 '2023-03-31',
 '2022-03-31',
 '2022-06-30',
 '2022-09-30',
 '2022-12-31',
 '2021-03-31',
 '2021-06-30',
 '2021-09-30',
 '2020-12-31',
 '2021-12-31',
 '2020-03-31',
 '2020-06-30',
 '2020-09-30',
 '2019-12-31',
 '2019-03-31',
 '2019-06-30',
 '2019-09-30',
 '2018-12-31',
 '2018-03-31',
 '2018-06-30',
 '2018-09-30',
 'b3_issuer_code',
 'equity',
 '2023-06-30',
 'QteTotalAcoes',
 '2023-09-30']

In [ ]:
import mysql.connector
import warnings
import os
warnings.filterwarnings('ignore')

db = mysql.connector.connect(
    user=os.getenv('MYSQL_USER'),
    password=os.getenv('MYSQL_PASSWORD'),
    host=os.getenv('MYSQL_HOST'),
    port=os.getenv('MYSQL_PORT'),
    database=os.getenv('MYSQL_DATABASE'),
    )

msql = pd.read_sql_query("SELECT * from cota_fundos", db)
msql = msql.reset_index(drop=True) 